<a href="https://colab.research.google.com/github/ahmedemaraengineer/ML/blob/main/30_Days_OF_ML(7).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns

plt.style.use("seaborn-whitegrid")

from sklearn.feature_selection import mutual_info_regression
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold

from sklearn.preprocessing import (OrdinalEncoder, StandardScaler, MinMaxScaler)

from sklearn.metrics import mean_squared_error, r2_score
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from scipy import stats
from scipy.stats import norm
from sklearn.linear_model import Lasso, RidgeCV
from sklearn.ensemble import (RandomForestRegressor, AdaBoostRegressor,
                             GradientBoostingRegressor, ExtraTreesRegressor)
from sklearn.ensemble import VotingRegressor

from lightgbm import LGBMRegressor
import lightgbm as lgb
from sklearn.svm import SVC 
from xgboost import XGBRegressor
%matplotlib inline

import warnings
warnings.filterwarnings('ignore')

In [ ]:
full_train = pd.read_csv("/content/gdrive/MyDrive/train.csv")

In [ ]:
full_train.head()

,id,cat0,cat1,cat2,cat3,cat4,cat5,cat6,cat7,cat8,cat9,cont0,cont1,cont2,cont3,cont4,cont5,cont6,cont7,cont8,cont9,cont10,cont11,cont12,cont13,target
0,1,B,B,B,C,B,B,A,E,C,N,0.201470,-0.014822,0.669699,0.136278,0.610706,0.400361,0.160266,0.310921,0.389470,0.267559,0.237281,0.377873,0.322401,0.869850,8.113634
1,2,B,B,A,A,B,D,A,F,A,O,0.743068,0.367411,1.021605,0.365798,0.276853,0.533087,0.558922,0.516294,0.594928,0.341439,0.906013,0.921701,0.261975,0.465083,8.481233
2,3,A,A,A,C,B,D,A,D,A,F,0.742708,0.310383,-0.012673,0.576957,0.285074,0.650609,0.375348,0.902567,0.555205,0.843531,0.748809,0.620126,0.541474,0.763846,8.364351
3,4,B,B,A,C,B,D,A,E,C,K,0.429551,0.620998,0.577942,0.280610,0.284667,0.668980,0.239061,0.732948,0.679618,0.574844,0.346010,0.714610,0.540150,0.280682,8.049253
4,6,A,A,A,C,B,D,A,E,A,N,1.058291,0.367492,-0.052389,0.232407,0.287595,0.686964,0.420667,0.648182,0.684501,0.956692,1.000773,0.776742,0.625849,0.250823,7.972260


In [ ]:
full_test = pd.read_csv("/content/gdrive/MyDrive/test.csv")

In [ ]:
num_cols = X_train.select_dtypes(include=['int64', 'float64']).columns
cat_cols = X_train.select_dtypes(include=['object', 'bool']).columns

In [ ]:
skewed_features = full_train[num_cols].apply(lambda x: stats.skew(x)).sort_values(ascending=False)
skewed_features = skewed_features[abs(skewed_features) > 0.75]
print(skewed_features)

for f in skewed_features.index:
    full_train[f] = np.log1p(full_train[f])
    full_test[f] = np.log1p(full_test[f])

cont4    0.776594
dtype: float64


In [ ]:
cat_transformer = OrdinalEncoder()
num_transformer = StandardScaler()
preprocessor = ColumnTransformer(transformers=[
    ('cat', cat_transformer, cat_cols),
    ('num', num_transformer, num_cols)])

In [ ]:
from sklearn.model_selection import train_test_split
X = full_train.drop(['target' ,'id'] ,axis = 1) # the independent variable
y = full_train.target # the dependent varaible
X_train ,X_valid ,y_train ,y_valid = train_test_split(X ,y ,train_size = 0.8)

In [ ]:
model = XGBRegressor(n_estimators= 10000, booster='gbtree', 
                    learning_rate= 0.03628302216953097, subsample= 0.7875490025178415, max_depth= 3,
                    colsample_bytree = 0.11807135201147481, reg_alpha = 23.13181079976304, random_state = 1,
                    reg_lambda = 0.0008746338866473539, n_jobs=4)
clf = Pipeline(steps=[('preprocessor', preprocessor),
                       ('model', model)])

In [ ]:
clf.fit(X_train ,y_train)

[17:05:16] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


Pipeline(memory=None,
         steps=[('preprocessor',
                 ColumnTransformer(n_jobs=None, remainder='drop',
                                   sparse_threshold=0.3,
                                   transformer_weights=None,
                                   transformers=[('cat',
                                                  OrdinalEncoder(categories='auto',
                                                                 dtype=<class 'numpy.float64'>),
                                                  Index(['cat0', 'cat1', 'cat2', 'cat3', 'cat4', 'cat5', 'cat6', 'cat7', 'cat8',
       'cat9'],
      dtype='object')),
                                                 ('num',
                                                  StandardScaler(copy=True,
                                                                 wi...
                              importance_type='gain',
                              learning_rate=0.03628302216953097,
                              m

In [ ]:
yhat = clf.predict(X_valid)
error = mean_squared_error(yhat ,y_valid ,squared = False)

In [ ]:
error

0.7123933844737658

In [ ]:
test = full_test.drop(['id'] ,axis = 1)
predictions = clf.predict(test)

In [ ]:
dic = {'id' : full_test.id ,'target' : predictions}
df = pd.DataFrame(dic)
df.head()

,id,target
0,0,8.044337
1,5,8.382872
2,15,8.429063
3,16,8.496202
4,17,8.127303
